In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

# Michael change
#from tensorflow import keras
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.5.0


Using TensorFlow backend.


# Preparing tables

### Bad stackoverflow

In [3]:
data1 = pd.read_csv("DB/stack-overflow-data.csv")

In [6]:
data1.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [11]:
data1.tags.unique()

array(['c#', 'asp.net', 'objective-c', '.net', 'python', 'angularjs',
       'iphone', 'ruby-on-rails', 'ios', 'c', 'sql', 'java', 'jquery',
       'css', 'c++', 'php', 'android', 'mysql', 'javascript', 'html'],
      dtype=object)

### Freelance

In [49]:
data2 = pd.read_csv("DB/Freelance_stackexange.csv")

In [50]:
data2.head()

,Id,Title,Body,Tags
0,1,"At what point is it no longer a ""Free Change""",<p>I am a freelance web developer. I have bui...,<contracts><changes><design>
1,2,Determining when to turn down business,<p>I recently started freelancing and found my...,<attracting-clients><project><difficult-client...
2,3,Tracking hours you are working for clients,"<p>When working through the day, I may work on...",<time-management><time-tracking>
3,6,How to deal with projects that have gone on to...,<p>I have a project where the client initially...,<project-management>
4,8,I failed to reach the deadline,<p>I started a project given by client and it ...,<time-management><project-management><communic...


In [21]:
import re

def body_cleaner(body):
    """body is a string"""
    body = re.sub(r'<[^>]*>', '', body)
    #body = re.sub(r'\^', '', body)
    return body

In [25]:
data2.Title[0]

'At what point is it no longer a "Free Change"'

In [24]:
body_cleaner(data2.Title[0])

'At what point is it no longer a "Free Change"'

In [51]:
freelance_data = pd.DataFrame(columns=['Body', 'Tag'])
freelance_tags = set()

In [52]:
str(['abc', 'cde'])

"['abc', 'cde']"

In [53]:
for body, tag_row in zip(data2.Body, data2.Tags):
    tags = (tag_row[1:-1]).split('><')
    freelance_tags |= set(tags)
    freelance_data = freelance_data.append({'Body' : body_cleaner(body), 'Tag' : tags}, ignore_index=True)

In [54]:
freelance_data.head()

,Body,Tag
0,I am a freelance web developer. I have built ...,"[contracts, changes, design]"
1,I recently started freelancing and found mysel...,"[attracting-clients, project, difficult-client..."
2,"When working through the day, I may work on ma...","[time-management, time-tracking]"
3,I have a project where the client initially re...,[project-management]
4,I started a project given by client and it wen...,"[time-management, project-management, communic..."


In [55]:
freelance_tags

{'accounting',
 'advertising',
 'attracting-clients',
 'billable-hours',
 'billing',
 'bootstrapping',
 'branch-office',
 'branding',
 'budget',
 'business-plan',
 'canada',
 'changes',
 'client-support',
 'clients',
 'communication',
 'company-management',
 'consulting',
 'contract-cancellation',
 'contract-to-hire',
 'contracts',
 'copyright',
 'customer-relations',
 'design',
 'difficult-client',
 'disability',
 'dress-code',
 'entity',
 'estimation',
 'ethics',
 'eu',
 'evaluating-clients',
 'expenses',
 'finance',
 'fixed-price',
 'foreign-incorporation',
 'foreign-markets',
 'france',
 'freelance-websites',
 'friends-services',
 'germany',
 'group-freelancing',
 'growth',
 'hmrc',
 'hourly',
 'incorporation',
 'information-technology',
 'insurance',
 'interview',
 'invoices',
 'late-payment',
 'legal',
 'licensing',
 'linkedin',
 'llc',
 'local-business',
 'long-term',
 'maintenance',
 'marketing',
 'money',
 'moonlighting',
 'nda',
 'negotiation',
 'nonprofit',
 'offer',
 'outso

In [56]:
freelance_data.to_csv("DB/freelance_data.csv", sep=',')

In [44]:
str(freelance_tags)

"{'ethics', 'productivity', 'disability', 'overtime', 'time-management', 'dress-code', 'virtual-office', 'attracting-clients', 'untagged', 'finance', 'referral', 'billing', 'startups', 'nonprofit', 'consulting', 'difficult-client', 'remote', 'money', 'maintenance', 'switzerland', 'design', 'invoices', 'uk', 'proposal', 'webmasters', 'translations', 'insurance', 'software', 'working-from-home', 'local-business', 'vat', 'ownership', 'company-management', 'nda', 'contract-cancellation', 'friends-services', 'trust', 'partner', 'freelance-websites', 'safety', 'hourly', 'sales', 'fixed-price', 'advertising', 'evaluating-clients', 'foreign-incorporation', 'project-bidding', 'accounting', 'copyright', 'contract-to-hire', 'share-ratio', 'regression-testing', 'changes', 'growth', 'client-support', 'price-per-hour', 'communication', 'contracts', 'payments', 'germany', 'long-term', 'entity', 'estimation', 'travel', 'profit-share', 'hmrc', 'small-projects', 'penalties', 'pay-rate', 'sickness', 'sco

In [57]:
out = open("DB/freelance_data_tags", 'w')

In [58]:
tr = len(freelance_tags) - 1
count = 0
for t in freelance_tags:
    out.write(t)
    if count < tr:
        count += 1
        out.write('|')
out.close()

### startup

In [59]:
data3 = pd.read_csv("DB/StartUp_stackexange.csv")

In [60]:
data3.head()

,Id,Title,Body,Tags
0,1,Is my LLC required to have a registered agent?,"<p>After registering my small LLC, I signed up...",<llc><new-hampshire><united-states>
1,2,Dividing profits amongst developers and designers,"<p>If I have a startup that produces apps, is ...",<tech-company><mobile-apps><equity>
2,6,How long should my business plan be?,"<p>In short, for my very first business plan w...",<business-plan>
3,7,What key intellectual property should a startu...,<p>We know that venture capitalists keep a kee...,<intellectual-property><investment><venture-ca...
4,13,Are there benefits gained from registering a l...,"<p>I'm thinking of doing some freelance work, ...",<united-kingdom><freelancing>


In [61]:
startup_data = pd.DataFrame(columns=['Body', 'Tag'])
startup_tags = set()

In [35]:
str(['abc', 'cde'])

"['abc', 'cde']"

In [63]:
for body, tag_row in zip(data3.Body, data3.Tags):
    tags = (tag_row[1:-1]).split('><')
    startup_tags |= set(tags)
    startup_data = startup_data.append({'Body' : body_cleaner(body), 'Tag' : tags}, ignore_index=True)

In [64]:
startup_data.head()

,Body,Tag
0,"After registering my small LLC, I signed up fo...","[llc, new-hampshire, united-states]"
1,"If I have a startup that produces apps, is the...","[tech-company, mobile-apps, equity]"
2,"In short, for my very first business plan whic...",[business-plan]
3,We know that venture capitalists keep a keen e...,"[intellectual-property, investment, venture-ca..."
4,"I'm thinking of doing some freelance work, and...","[united-kingdom, freelancing]"


In [65]:
startup_tags

{'83b',
 'ab-testing',
 'accelerators',
 'accounting',
 'acquisition',
 'advertising',
 'affiliate-marketing',
 'agile',
 'analytics',
 'android-development',
 'angel-investors',
 'app-store',
 'app-store-optimization',
 'applications',
 'applying',
 'australia',
 'b2b',
 'benefits',
 'beta-testing',
 'billing',
 'bizspark',
 'board',
 'bootstrapping',
 'brand',
 'budget',
 'buisness-model',
 'business-capital',
 'business-card',
 'business-methodology',
 'business-model',
 'business-plan',
 'business-proposal',
 'business-registration',
 'business-structure',
 'buy-out',
 'c-corporation',
 'california',
 'canada',
 'cashflow',
 'ceo',
 'chicken-egg',
 'china',
 'churn',
 'closing',
 'cloud',
 'co-founder',
 'colorado',
 'commissions',
 'communication',
 'community',
 'company-culture',
 'compensation',
 'competition',
 'competitive-events',
 'consulting',
 'content',
 'contractors',
 'contracts',
 'convertible-note',
 'copyright',
 'corporation',
 'crm',
 'crowd-funding',
 'cto',
 'cu

In [66]:
startup_data.to_csv("DB/startup_data.csv", sep=',')

In [68]:
out = open("DB/startup_data_tags", 'w')

In [69]:
tr = len(startup_tags) - 1
count = 0
for t in startup_tags:
    out.write(t)
    if count < tr:
        count += 1
        out.write('|')
out.close()